## 常见的优化有以下几个方面：
* SQL及索引
* 数据库表结构
* 系统配置
* 硬件
<br/>
上面这些越来越成本高，但是优化效果越来越差。

# 一、SQL及索引的优化

### 如何发现有问题的SQL？
使用MYSQL慢查询日志对有效率问题的SQL进行监控：
* `show variables like 'slow_query_log'`
* `set global slow_query_log_file='/home/mysql/sql_log/mysql-slow.log'`这个是指定了慢查询的日志的位置在哪里。
* `set global log_queries_not_using_indexs=on;`指定是否把没有使用索引的SQL记录到慢查询日志中，通常是需要的。
* `set global long_query_time=1`指定把大于多少秒的SQL记录到慢查询日志中。

#### 慢查询日志包含的内容
* 执行SQL的主机信息:# User@Host:root[root]@localhost[]
* SQL的执行信息: # Query_time:0.00024 Lock_time:0.000000 Rows_sent:0 Rows_examined:0
* SQL执行时间: SET timestamp=1408275856;
* SQL的内容: select CONCAT();

#### 慢查日志的分析工具
* mysqldumpslow(MySQL官方的，会随着MySQL自动安装到我们的服务器中， -h来显示说明)
* pt-query-digest(慢查日志的分析工具)
    * 输出到文件: pt-query-digest slow.log > slow_log.report
    * 输出到数据库表: pt-query-digest slow.log -review h=127.0.0.1,D=test,p=root,P=3306,u=root,t=query_review --create-reviewtable --review-hostory t=hostname_slow。其中的`review`参数是把结果放单数据库的一个表里。通过`create-reviewtable`来建立相关的存储表。
    * 输出的内容包含了三个部分：
        1. 头信息，显示出了日志的时间范围，以及总的SQL数量和不同的SQL数量。
        2. `ResponseTime`:响应时间占比，`Calls`:sql执行次数。
        3. 具体的SQL是什么。

#### 如何通过慢查日志发现有问题的SQL。
1. **查询次数多且每次查询占用时间长的SQL。**通常为pt-query-digest分析的前几个查询。
2. **IO大的SQL。**注意pt-query-digest分析中的`Rows examine`项。
3. **未命中索引的SQL。**注意pt-query-digest分析中`Rows examine`(表示扫描的行数)和`Rows Send`(表示发送的行数)的对比。

#### 通过explain查询和分析SQL的执行计划
通过explain `[我们自己的SQL]`来看执行的计划。
`explain`返回各列的含义：
* `table`:显示这一行的数据是关于哪张表的；
* `type`:这是重要的列，显示连接使用了何种类型。**从最好到最差的连接类型为`const`(表示是长度查找，一般是主键查找)、`eq_reg`(范围查找，一般是唯一索引和主键的 范围查找)、`ref`(常见于连接的查询中，基于某一个索引查找)、`range`(基于索引的范围查找)、`index`(通常是对于索引的扫描)和`ALL`(实际上是进行表扫描)。**
* `possible_keys`:显示可能应用在这张表中的索引。如果为空，没有可能的索引。
* `key`: 实际使用的索引。若果为`NULL`，则没有使用索引。
* `key_len`: 使用的索引的长度。在不损失精确性的情况下，长度越短越好。
* `ref`: 显示索引的哪一列被使用了，如果可能的话，是一个常数。
* `rows`: MySQL认为必须检查的用来返回请求数据的行数。
* `extra`：列需要注意的返回值：
    * `Using filesort`：看到这个的时候，查询就需要优化了。MYSQL需要进行额外的步骤来发现如何对返回的行排序。它根据连接类型以及存储排序键值和匹配条件的全部行的指针来排序全部行。
    * `Using temporary`：看到这个的时候，查询需要优化了。这里，MYSQL需要创建一个临时表来存储结果，这通常发生在对不同的列集进行`ORDER BY`上，而不是`GROUP BY`上。

#### `Count()`和`Max()`的优化
* `Max`可以通过建立索引（`create index idx_paydate[索引的名称] on payment[表名](payment_date[列名]);`来优化。 **需要注意的是索引是顺序排序的！**）完全能够通过索引能查到我们需要的结果，这种索引叫做`覆盖索引`。
* `Count`函数中，实际上也可以加判断条件，如:Count(release_year=2006 Or null)。`需要注意的是Count(*)是包括空值的，而Count(id)是不包含空值的`。

#### 子查询的优化
* 通常情况下，需要把子查询优化为join查询，但在优化时要注意**关联键是否有一对多的关系，要注意重复数据(需要用到`distinct`)**。

#### group by的优化
* 如果有查询条件的话，尽量在子查询中进行，这样不会生成临时表。

#### Limit查询的优化
* limit常用于分页处理，时常会伴随`order by`从句使用，因此大多时候会使用`Filesorts`，这样会造成大量的IO问题。
* 优化步骤1：使用`有索引的列`或`主键`进行`Order by`操作。
* 优化步骤2：记录上次返回的主键，在下次查询时使用主键过滤。(避免了数据量大时扫描过多的记录。**这种方式要求主键是顺序排列的，是顺序增长的。对于不连续的，我们需要建立附加的列。**)

# 二、索引优化

#### 如何选择合适的列建立索引？
1. 在`where`从句，`group by`从句，`order by`从句，`on`从句中出现的列(这种情况的索引叫做覆盖索引)；
2. 索引字段越小越好；
3. 离散度大的列放到联合索引的前面(**通过Count查询唯一值，唯一值越多，离散度越好**)。

#### 索引优化SQL的方法
**索引能增查询的速度，但是会降低写入的速度。**<br/>
1. 索引的维护及优化——重复及冗余索引：`重复索引`是指相同的列以相同的顺序建立的同类型的索引(例如给主键增加unique)。`冗余索引`是指多个索引的前缀列相同，或是在联合索引中包含了主键的索引。
2. 索引的维护及优化——查找重复及冗余索引：(1)要用到`information_schema`来进行查看。(2)使用`pt-duplicate-key-checker`工具检查重复及冗余索引。(**pt-duplicate-key-checker -uroot[用户] -p '123456'[密码] -h 127.0.0.1[服务器端口]**)
3. 索引的维护及优化——删除不用索引：目前MySQL中还没有记录索引的使用情况，但是在`PerconMySQL`和`MariaDB`中可以通过`INDEX_STATISTICS`表来查看哪些索引未使用，但在MySQL中目前只能通过慢查询日志配合`pt-index-usage`工具来进行索引使用情况的分析。(`pt-index-usage -uroot -p'1234' mysql-slow.log`)

# 三、数据库结构优化

### 选择合适的数据类型
数据类型的选择，重点在于**合适**二字，如何确定选择的数据类型是否合适？
1. 使用可以存下你的数据的最小的数据类型。(使用int来存储日期时间，利用`FROM_UNIXTIME()【进行转换】,UNIX_TIMESTAMP()【存的时候用】两个函数来进行转换。`；使用`bigint`来IP地址，利用`INET_ATON()【IP转换成bigint】,INET_NTOA()【bigint转换成IP】`两个函数来进行转换。**这些更省空间，更加高效。**)
2. 使用简单的数据类型。Int要比VARChAR类型在mysql处理上简单。
3. 尽可能的使用`not null`定义字段。(非not null会需要特殊的字段金习性存储，并且增加IO的操作。)
4. 尽量少用`text`类型，费用不可时最好考虑分表。

### 表的范式化优化
#### 表的范式化和反范式化
`范式化`是指数据库设计的规范，目前说到范式化一般是指第三设计范式，也就是**要求数据表中不存在非关键字段对任意候选关键字段的传递函数依赖则符合第三范式**。


商品|价格|重量|有效期|分类|分类描述
:-:|:-:|:-:|:-:|:-:|:-:
可乐|3.00|250ml|2014.6|饮料|碳酸饮料
北冰洋|3.00|250ml|2014.7|饮料|碳酸饮料
<br/>
存在以下传递函数依赖关系：<br/>
(商品名称)->（分类）->(分类描述)<br/>
也就是说存在非关键字段“分类描述”对关键字段“商品名称”的传递函数依赖。

不符合第三范式要求的表存在下列问题：
1. 数据冗余:(分类，分类描述)对于每一个商品都会进行记录；
2. 数据的插入异常；
3. 数据的更新异常；
4. 数据的删除异常

#### 数据库表的反范式化优化
`反范式化`是指为了 查询效率的考虑把原来符合第三范式的表**适当**的增加冗余，以达到优化 查询效率的目的，**反范式化是一种以空间来换取时间的操作**。

### 表的垂直拆分
所谓的垂直拆分，就是把原来一个有很多列的表拆分成多个表，**这解决了表的宽度问题**。通常垂直拆分可以按一下原则进行：
1. 把不常用的字段单独存放到一个表中。
2. 把大字段独立存放到一个表中。
3. 把经常一起使用的字段放到一起。

### 表的水平拆分
表的水平拆分是为了解决单表的数据量过大的问题，水平拆分的表每一个表的结构都是完全一致的。<br/>
导致的挑战：
1. 跨分区表进行数据查询
2. 统计及后台报表操作

# 四、系统配置优化

### 操作系统配置优化
数据库试试基于操作系统的，目前大多数MySQL都是安装在Linux系统上，所以对于操作西永的一些参数配置也会影响到MySQL的性能，下面就列出一些常用到的系统配置。
1. 网络方面的配置，要修改`/etc/sysctl.conf`文件
    1.1 #增加tcp支持的对列数 net.ipv4.tcp_max_syn_backlog = 65535
    1.2 # 减少断开连接时，资源回收：net.ipv4.tcp_max_tw_buckets = 8000;   net.ipv4.tcp_tw_reuse = 1;   net.ipv4.tcp_tw_recycle = 1; net.ipv4.tcp_fin_timeout = 10
2. 打开文件数的限制，可以使用`ulimit -a`查看目录的各种限制，可以修改`/etc/security/limits.conf`文件，增加以下内容以修改打开文件数量的限制: * soft nofile 65535 ;  * hard nofile 65535。
    2.1  除此之外，最好在MySQL服务器上关闭`iptables,selinux`等防火墙软件。
    
### MySQL配置文件
MySQL可以通过启动时指定配置参数和使用配置文件两种方式进行配置，在大多数情况下配置文件位于`/etc/my.cnf`或是`/etc/mysql/my.cnf`在windows系统配置文件可以是位于`C:/windows/my.ini`文件，MySQL查找配置文件的顺序可以通过一下方法获得`$/usr/sbin/mysqld --verbose --help | grep -A 1 'Default options'`。**注意：如果存在多个位置㛮配置文件，则后面的会覆盖前面的。**
#### MySQL配置文件——常用参数说明
* `innodb_buffer_pool_size`:非常重要的一个参数，用于配置`Innodb`的缓冲池，如果数据库中只有`Innodb`表，则推荐配置量为总内存的75%。
* `innodb_buffer_pool_instances`:MySQL5.5中新增加参数，可以控制缓冲池的个数，默认情况下只有一个缓冲池。
* `innodb_log_buffer_size`:innodb log缓冲的大小，由于日志最长每秒钟就会刷新，所以一般不会太大。
* `innodb_flush_log_at_trx_commit`:关键参数(**多长时间把变更刷新到磁盘**)，对innodb的IO效率影响很大。默认值为1，可以取0，1，2三个值，一般建议设为2，但如果数据安全性要求比较高则使用默认值1。
* `innodb_read_io_threads`和`innodb_write_io_threads`:以上两个参数决定了Innodb读写的IO进程数，默认为4。
* `innodb_file_per_table`:关键参数，控制Innodb每一个表使用独立的表空间，默认为OFF，也就是所有表都会建立在共享表空间中。
* `innodb_stats_on_metadata`:决定了MySQL在什么情况下会刷新innodb表的统计信息。最好设置为OFF。

### 第三方配置工具
#### Percon Configuration Wizard(https://tools.percona.com/wizard[https://tools.percona.com/wizard])

# 五、服务器硬件优化

### 如何选择CPU
思考：是选择单核更快的CPU还是选择核数更多的CPU？<br/>
1. MySQL有一些工作只能使用到单核CPU：Replicate，SQL。。。
2. MySQL对CPU核数的支持并不是越多越快。MySQL5.5使用的服务器不要超过32核。
### Disk IO优化
常用RAID界别简介<br/>
* RAID0:也称为条带，就是把东哥磁盘链接成一个 银盘使用，这个级别IO最好。
* RAID1：也称为镜像，要求至少有两个磁盘，每组磁盘存储的数据相同。
* RAID5：也是把多个(最少3个)银盘合并成一个逻辑盘使用，数据读写时会建立奇偶校验信息，并且奇偶校验信息和相对应的 数据分别存储 于不同的磁盘上。当RAID5的一个磁盘数据发生损坏后，利用剩下的数据和相应的奇偶校验信息去恢复被损坏的数据。
* RAID1+0：就是RAID1和RAID0的结合。同时具备两个级别的 优缺点。**一般建议数据库使用这个级别。**


思考：SNA和NAT是否适合数据库？
1. 常用于高可用解决方案。
2. 顺序读写效率很高，但是随即单独写不如人意。
3. 数据库随机读写比率很高。